In [ ]:
from odf.sbe.io import read_hex
import odf.sbe.accessors
import numpy as np
import xarray as xr

In [ ]:
# %%time
ds = read_hex("00102.hex")

### Frequency testing

In [ ]:
# The accessor lets you get frequencies like they are an attribtue on sbe
ds.sbe.f0

In [ ]:
# just change the channel number
ds.sbe.f3

In [ ]:
# There are no guard rails right now
ds.sbe.f10

In [ ]:
# volts work the same way
ds.sbe.v0

In [ ]:
ds.sbe.v7

In [ ]:
ds

### XMLCON testing

In [ ]:
import odf.sbe.parsers as parsers

In [ ]:
xmlcon_box, xmlcon_sensors = parsers.parse_xmlcon(ds.xmlcon)

In [ ]:
ds.xmlcon

In [ ]:
xmlcon_box

In [ ]:
xmlcon_sensors.keys()

In [ ]:
ds.bl

In [ ]:
bl, a2 = parsers.parse_bl(ds.bl.item())

In [ ]:
type(a2)


### META accessors

In [ ]:
import odf.sbe.channels as channels

In [ ]:
meta_out = channels.metadata_wrapper(ds.hex, xmlcon_box)

In [ ]:
meta_out

In [ ]:
x = xr.Dataset({
    channel: ds.sbe[channel] for channel in ds.sbe()
})

In [ ]:
xr.Dataset({channel: ds.sbe[channel] for channel in ds.sbe})

In [ ]:
print(list(ds.sbe))

In [ ]:
xr.Dataset({channel: ds.sbe()[channel] for channel in ds.sbe()})

In [ ]:
print(ds)

In [ ]:
print(ds.sbe)

In [ ]:
%%time
ds.sbe.meta

In [ ]:
print(ds.sbe.meta)

In [ ]:
def metadata_wrapper(hex_data, cfg):
    """
    A wrapper of sorts to handle columnar metadata in the source HEX file.
    """

    f_s, v_s = (
        cfg[i] for i in ["FrequencyChannelsSuppressed", "VoltageWordsSuppressed"]
    )
    num_f = 5 - f_s
    num_v = 8 - v_s
    start_byte_ix = int(num_f * 3 + num_v * 1.5)

    ix_tracker = start_byte_ix

    # Initialize an xarray Dataset instead of a DataArray
    meta_out = xr.Dataset()
    meta_out["scan_number"] = xr.DataArray(
        np.arange(1, len(hex_data) + 1), dims=["scan"]
    )

    if cfg["NmeaPositionDataAdded"]:
        col_extracts = hex_data[:, ix_tracker : ix_tracker + 7].astype(int)
        data_to_write = channels._nmeaposition(col_extracts)
        ix_tracker += 7

        for var_name, values in zip(data_to_write.coords["variable"].values, data_to_write.T.values):
            meta_out[var_name] = xr.DataArray(values, dims=["scan"])

    if cfg["NmeaTimeAdded"]:
        col_extracts = hex_data[:, ix_tracker : ix_tracker + 4]
        data_to_write = channels._sbe_time(col_extracts, "nmea")
        ix_tracker += 4

        meta_out["nmea_timestamp"] = xr.DataArray(data_to_write.values[:, 0], dims=["scan"])

    col_extracts = hex_data[:, ix_tracker : ix_tracker + 3]
    data_to_write = channels._sbe9core(col_extracts)
    ix_tracker += 3

    for var_name, values in zip(data_to_write.coords["variable"].values, data_to_write.T.values):
        meta_out[var_name] = xr.DataArray(values, dims=["scan"])

    if cfg["ScanTimeAdded"]:
        col_extracts = hex_data[:, ix_tracker : ix_tracker + 4]
        data_to_write = channels._sbe_time(col_extracts, "scan")
        ix_tracker += 4

        meta_out["scan_timestamp"] = xr.DataArray(data_to_write.values[:, 0], dims=["scan"])
    else:
        print("No ScanTimeAdded in XMLCON - building manually from frequency.")

    if ix_tracker > len(hex_data[0]):
        print(
            f"Uh oh, might not have incremented metadata correctly."
            f" Got {ix_tracker} bytes out of a possible {len(hex_data[0])}."
        )

    return meta_out

In [ ]:
x = metadata_wrapper(ds.hex, xmlcon_box)

In [ ]:
x